In [ ]:
from pyscipopt import Model
# -------------------- # Mengen # --------------------T = ["T1", "T2", "T3"]
D = ["Diesel_Actros"]
E = ["E400", "E600"]

N = 260c_maut = 0.34# -------------------- # Parameter # --------------------d_maut = {"T1": 120, "T2": 80, "T3": 150}

c_capex_d = {"Diesel_Actros": 90000}
c_opex_d = {"Diesel_Actros": 15000}
c_tax_d  = {"Diesel_Actros": 3000}

c_capex_e = {"E400": 120000, "E600": 140000}
c_opex_e  = {"E400": 12000, "E600": 13000}
r_THG     = {"E400": 3000, "E600": 3500}
# -------------------- # Modell # --------------------model = Model("Fleet_Assignment_Separated")
# Variablenx_d = {d: model.addVar(vtype="INTEGER", lb=0, name=f"x_d_{d}") for d in D}
x_e = {e: model.addVar(vtype="INTEGER", lb=0, name=f"x_e_{e}") for e in E}

a_td = {(t, d): model.addVar(vtype="BINARY", name=f"a_{t}_{d}") for t in T for d in D}
a_te = {(t, e): model.addVar(vtype="BINARY", name=f"a_{t}_{e}") for t in T for e in E}
# Zielfunktionmodel.setObjective(
    sum(x_d[d] * (c_capex_d[d] + c_opex_d[d] + c_tax_d[d]) for d in D)
    + sum(x_e[e] * (c_capex_e[e] + c_opex_e[e]) for e in E)
    + N * sum(a_td[t, d] * d_maut[t] * c_maut for t in T for d in D)
    - sum(x_e[e] * r_THG[e] for e in E),
    "minimize")
# Nebenbedingungen for t in T:
    model.addCons(
        sum(a_td[t, d] for d in D) +
        sum(a_te[t, e] for e in E) == 1    )
for d in D:
    model.addCons(sum(a_td[t, d] for t in T) <= x_d[d])
for e in E:
    model.addCons(sum(a_te[t, e] for t in T) <= x_e[e])
# Optimierenmodel.optimize()

In [ ]:
print("Diesel-Lkw:")for d in D:
    print(d, int(model.getVal(x_d[d])))
print("\nElektro-Lkw:")for e in E:
    print(e, int(model.getVal(x_e[e])))
print("\nTourenzuordnung:")for t in T:
    for d in D:
        if model.getVal(a_td[t, d]) > 0.5:
            print(f"{t} -> {d}")
    for e in E:
        if model.getVal(a_te[t, e]) > 0.5:
            print(f"{t} -> {e}")
print("\nGesamtkosten:", model.getObjVal())